In [40]:
import pandas as pd
import numpy as np
from PIL import Image

### 实现image与pandas矩阵互转

In [41]:
## 这个函数将图片转为pandas灰度矩阵
def grayscale_to_matrix(image_path):
    # 打开图像
    image = Image.open(image_path)
    # 将图像转换为灰度图像
    grayscale_image = image.convert("L")
    # 转换为 numpy 数组
    image_np = np.array(grayscale_image)
    #转换为pandas矩阵
    image_df=pd.DataFrame(image_np)
    return image_np,image_df

##这个函数将矩阵转换为图片，传入一个pandas矩阵
def matrix_to_image(matrix,name):
    # 将 Pandas DataFrame 转换为 NumPy 数组
    image_array = matrix.to_numpy(dtype='uint8')
    # 创建 Image 对象
    image = Image.fromarray(image_array)
    # 保存图像为 BMP 文件
    image.save(name)
    return 

### 实现csv文件与pandas矩阵互转

In [42]:
def to_csvs(df):
    df.to_csv("out.csv",index=False, header=False)#不保存行索引和列索引
    
def csv_to_matrix(file_path):
    # 从CSV文件加载数据并转换为Pandas DataFrame
    matrix = pd.read_csv(file_path, header=None)#header=None是说不要将第一行转换为列名
    return matrix

### 将大图像切成若干图像块

In [43]:
# n1:图像块的高
# n2:图像块的宽
def split_matrix(matrix,n1,n2):
    # 计算矩阵的行数和列数
    rows, cols = matrix.shape
    # 创建一个空的字典来存储子矩阵
    sub_matrices = {}
    
    # 将矩阵分割为若干个 n1*n2 的子矩阵
    index = 1
    for i in range(0, rows, n1):
        for j in range(0, cols, n2):
            sub_matrix = matrix.iloc[i:i+n1, j:j+n2]
            sub_matrices[index] = sub_matrix
            index += 1
    return sub_matrices

### 将若干图像块矩阵拼成一个大图像矩阵

In [44]:
# 定义最开始的大图像矩阵被分割成“m1*m2”个小矩阵
# m1是指大图像矩阵一行上被分割为多少个小矩阵,取决于图像块block的大小

def combine_dataframes(dictionary,m1):
    # 按照键的大小顺序对字典中的 DataFrame 进行排序
    sorted_dfs = sorted(dictionary.items(), key=lambda x: x[0])
    
    # 将排序后的 DataFrame 以每 m1 个一组的方式拼接成一行
    rows = [pd.concat([df for _, df in sorted_dfs[i:i+m1]], axis=1) for i in range(0, len(sorted_dfs), m1)]
    
    # 将拼接后的每行 DataFrame 组合成一个大的 DataFrame
    result_df = pd.concat(rows, axis=0)
    
    return result_df


### 将图像块的灰度矩阵转化为灰度值序列

In [45]:
def matrix_to_dict(matrix):
    # 将 Pandas DataFrame 转换为 NumPy 数组
    array = matrix.to_numpy()
    # 创建一个空的字典
    result_dict = {}    #result_dict记录的是原始顺序的灰度矩阵信息，以“原始序号：灰度值”的形式存储
    # 获取矩阵的行数和列数
    rows, cols = array.shape
    # 遍历矩阵中的每个元素，并按顺序编号存储到字典中
    index = 1
    for i in range(rows):
        for j in range(cols):
            result_dict[index] = array[i][j]
            index += 1
    #sorted_dict记录的是排序顺序的灰度矩阵信息，以“原始序号：灰度值”的形式存储
    sorted_list = sorted(result_dict.items(), key=lambda x: x[1], reverse=True)
    return result_dict,sorted_list

### 将灰度值序列转化为图像块灰度矩阵

In [46]:
# n1:图像块的高
# n2:图像块的宽
def list_to_dataframe(data,n1,n2):
    # 生成一个2x2的空DataFrame
    matrix = pd.DataFrame(index=range(n1), columns=range(n2))
    
    # 按照元组的第一个元素排序
    sorted_data = sorted(data, key=lambda x: x[0])
    
    # 逐个填充到DataFrame中
    for idx, (i, j) in enumerate(sorted_data):
        matrix.iloc[idx // n2, idx % n2] = j
    
    return matrix

### 计算e_max

In [47]:
def calculate_e_max(block_matrix):#传入一个图像块的灰度矩阵
    #将灰度矩阵的像素点按灰度值大小排序
    #sorted_list是这样的：[(2, 199), (4, 196), (1, 65), (3, 54)]
    #以(2, 199)为例，2是像素点排序前的的原始编号，199是像素点的灰度值
    _,sorted_list=matrix_to_dict(block_matrix) 
    e_max=sorted_list[0][1]-sorted_list[1][1]
    return e_max

### 找到一个DataFrame中的最大值以及其对应的逻辑索引

In [48]:
def find_max_value_and_position(dataframe):
    # 找到 DataFrame 中的最大值
    max_value = dataframe.max().max()
    # 找到最大值所在的行和列
    row_idx, col_idx = dataframe.stack().idxmax()
    return row_idx, col_idx

### 二进制序列与文字序列互转

In [49]:
def binary_to_text(binary_sequence):
    # 将二进制序列按照每8位拆分为一个字节
    bytes_list = [binary_sequence[i:i+8] for i in range(0, len(binary_sequence), 8)]
    # 将每个字节转换为对应的 ASCII 字符
    text = ''.join(chr(int(byte, 2)) for byte in bytes_list)
    return text

def text_to_binary(text):
    # 将字符串转换为 ASCII 编码的字节序列
    bytes_text = text.encode('ascii')
    # 将字节序列转换为二进制字符串
    binary_text = ''.join(format(byte, '08b') for byte in bytes_text)
    bit_list = [int(bit) for bit in binary_text]
    return binary_text,bit_list

# 主函数

### 加密部分

In [50]:
image_np,image_df=grayscale_to_matrix("Airplane.bmp")

In [51]:
#打印载入的灰度矩阵
#image_df

In [52]:
#图像切割成若干2*2的图像块
sub_matrices=split_matrix(image_df,2,2)
#打印切割的图像块以检查分割情况
sub_matrices[1]

,0,1
0,65,199
1,54,196


In [53]:
#计算每个图像块的e_max
sub_blocks={}#这个与sub_matrices的区别在于它记录了e_max的信息
for (i,j) in zip(sub_matrices.keys(),sub_matrices.values()):
    e_max=calculate_e_max(j)
    sub_blocks[(i,e_max)]=j

In [54]:
#计算这张图片的最大嵌入信息量
max_count=0
for i in sub_blocks.keys():
    if i[1]==1:
        max_count+=1
print("最大嵌入信息量为："+str(max_count)+" bit")

最大嵌入信息量为：20070 bit


In [55]:
##开始信息嵌入
secret_text="Poetry and melody are eternal romance. ----Victor Hugo"
_ ,data_1=text_to_binary(secret_text) #将加密信息转化为二进制list
data_2=[0] * ( max_count-len(data_1) )  #因为一共有20070位嵌入数据，因此还要设定一下填充序列
data=data_1+data_2
#print(data)

cnt=0
for i,j in zip(sub_blocks.keys(),sub_blocks.values()):
    if i[1]==1:
        row,col=find_max_value_and_position(j) #定位到改block的灰度值最大的像素点，进行灰度值调整
        sub_blocks[i].loc[row][col]=j.loc[row][col]+data[cnt] 
        cnt+=1
    if i[1]>1:
        row,col=find_max_value_and_position(j)
        sub_blocks[i].loc[row][col]=j.loc[row][col]+1 

sub_blocks_final={}
for i,j in zip(sub_blocks.keys(),sub_blocks.values()):
    sub_blocks_final[i[0]]=j        

In [56]:
##将修改后的图像块拼成一个大的
modified_matrix=combine_dataframes(sub_blocks_final,256) #512*512的图像被2*2的block分割，一行可以分出256个block
#将加密图像还原
matrix_to_image(modified_matrix,"secret_image.bmp")

### 解密部分

In [57]:
image_np,image_df=grayscale_to_matrix("secret_image.bmp")

In [58]:
#图像切割成若干2*2的图像块
sub_matrices=split_matrix(image_df,2,2)
#打印切割的图像块以检查分割情况
sub_matrices[1]

,0,1
0,65,200
1,54,196


In [59]:
#计算每个图像块的e_max，这里是修改后的e_max
sub_blocks={}#这个与sub_matrices的区别在于它记录了e_max的信息
for (i,j) in zip(sub_matrices.keys(),sub_matrices.values()):
    e_max=calculate_e_max(j)
    sub_blocks[(i,e_max)]=j

In [60]:
#开始信息提取，并恢复原来的图像
massage_binary=""
for i,j in zip(sub_blocks.keys(),sub_blocks.values()):
    if i[1]==1:
        massage_binary=massage_binary+"0"
    if i[1]==2:
        massage_binary=massage_binary+"1"
        row,col=find_max_value_and_position(j) #定位到改block的灰度值最大的像素点，进行灰度值调整
        sub_blocks[i].loc[row][col]=j.loc[row][col]-1 
    if i[1]>2:
        row,col=find_max_value_and_position(j) #定位到改block的灰度值最大的像素点，进行灰度值调整
        sub_blocks[i].loc[row][col]=j.loc[row][col]-1 
    
sub_blocks_final={}
for i,j in zip(sub_blocks.keys(),sub_blocks.values()):
    sub_blocks_final[i[0]]=j    

In [61]:
## 查看加密信息
message=binary_to_text(massage_binary)
print(message)

Poetry and melody are eternal romance. ----Victor Hugo                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [62]:
##将更新后的图像块拼成一个大的
modified_matrix=combine_dataframes(sub_blocks_final,256) #512*512的图像被2*2的block分割，一行可以分出256个block
## 将还原的图像保存
matrix_to_image(modified_matrix,"Restored_image.bmp")

### PSNR计算

In [63]:
def calculate_psnr(original, reconstructed):
    # 计算均方误差
    mse = np.mean((original - reconstructed) ** 2)
    # 计算PSNR
    max_val = 255  # 8位灰度图像的最大像素值
    PSNR = 10 * np.log10((max_val ** 2) / mse)
    return PSNR

def calculate_mse(original, reconstructed):
    # 计算均方误差
    mse = np.mean((original - reconstructed) ** 2)
    return mse

original,_=grayscale_to_matrix("Airplane.bmp")
modified,_=grayscale_to_matrix("secret_image.bmp")
restored,_=grayscale_to_matrix("Restored_image.bmp")

psnr_value = calculate_psnr(original, modified)
mse_value=calculate_mse(original,restored)

print("PSNR :", psnr_value)
print("MSE :", mse_value)

PSNR : 57.55353981561058
MSE : 0.0
